# Data Processing and Prep Work

First, we want to import all related libraries.

In [188]:
import pandas as pd
import numpy as np

Then, we want to read data from csv files provided.

In [189]:
results = pd.read_csv("Games.csv", usecols=["2","4","5","7"])
teams = pd.read_csv("Teams.csv", header=None)

In [190]:
results

,2,4,5,7
0,37,84,314,76
1,267,76,6,39
2,39,86,63,56
3,41,57,33,55
4,134,83,42,43
...,...,...,...,...
5369,73,89,22,71
5370,113,66,27,61
5371,127,61,298,51
5372,178,80,304,55


As we can see in the current results table, the data is messy, so we want to do data cleaning, especially through dropping unnecessary columns and rename columns for better interpretation in both dataFrames.

In [191]:
# rename columns
results.rename(columns={"2": "Team_win", "4": "Score_win", "5": "Team_lose", "7": "Score_lose"}, inplace=True)
results

,Team_win,Score_win,Team_lose,Score_lose
0,37,84,314,76
1,267,76,6,39
2,39,86,63,56
3,41,57,33,55
4,134,83,42,43
...,...,...,...,...
5369,73,89,22,71
5370,113,66,27,61
5371,127,61,298,51
5372,178,80,304,55


The results dataFrame has been really clear after preprocessing. We want to do it for the teams dataFrame as well.

In [192]:
teams

,0,1
0,1,Abilene_Chr
1,2,Air_Force
2,3,Akron
3,4,Alabama
4,5,Alabama_A&M
...,...,...
356,357,Wright_St
357,358,Wyoming
358,359,Xavier
359,360,Yale


In [193]:
#rename team columns
columns = teams.columns
teams.drop([columns[0]], axis=1, inplace=True)
teams.rename(columns={columns[1]:"Team_name"}, inplace=True)
teams

,Team_name
0,Abilene_Chr
1,Air_Force
2,Akron
3,Alabama
4,Alabama_A&M
...,...
356,Wright_St
357,Wyoming
358,Xavier
359,Yale


In [194]:
results[results["Team_lose"]==0]

,Team_win,Score_win,Team_lose,Score_lose


We notice that the id of team in the res Dataframe start from 1 so we will subtract 1 from the ids so that it matches the other dataFrame (as the index starts from 0).

In [195]:
# currently are the team numbers in the dataFrame are strings
# convert the strings into integers for subtraction
results["Team_win"] = results["Team_win"].astype(int)
results["Team_lose"] = results["Team_lose"].astype(int)

results["Team_win"] = results["Team_win"] - 1
results["Team_lose"] = results["Team_lose"] - 1
results

,Team_win,Score_win,Team_lose,Score_lose
0,36,84,313,76
1,266,76,5,39
2,38,86,62,56
3,40,57,32,55
4,133,83,41,43
...,...,...,...,...
5369,72,89,21,71
5370,112,66,26,61
5371,126,61,297,51
5372,177,80,303,55


Now, we have completed data preprocessing and cleaning.

# Colley's Method

In his formulation, Colley defines $\widehat{s}_{1}, \cdots, \widehat{s}_{k}$ by the matrix equation

$$
\left(\begin{array}{ccccc}
n_{1 \bullet}+2 & -n_{12} & -n_{13} & \cdots & -n_{1 k} \\
-n_{21} & n_{2 \bullet}+2 & -n_{23} & \cdots & -n_{2 k} \\
\vdots & \vdots & \ddots & \cdots & \vdots \\
-n_{k 1} & -n_{k 2} & \cdots & -n_{k, k-1} & n_{k}+2
\end{array}\right)\left(\begin{array}{c}
\widehat{s}_{1} \\
\widehat{s}_{2} \\
\vdots \\
\widehat{s}_{k}
\end{array}\right)=\left(\begin{array}{c}
W_{1}+1-n_{1 \bullet} / 2 \\
W_{2}+1-n_{2 \bullet} / 2 \\
\vdots \\
W_{k}+1-n_{k \bullet} / 2
\end{array}\right)
$$

where

$$
\begin{aligned}
k &= \text { number of teams } ,\\
W_{i} & =\text { number of wins for team } i, \\
n_{i j}=n_{j i} & =\text { number of games between teams } i \text { and } j, \text { and } \\
n_{i \bullet}=\sum_{j \neq i} n_{i j} & =\text { total number of games played by team } i .
\end{aligned}
$$

In our code, we defined $C$ is the $k \times k$ matrix and $S$ is the final vector

In [196]:
k = len(teams.index)
print("There are", k, "teams in total.")

There are 361 teams in total.


In [197]:
# initialize DataFrame with 0s in all entries
C = pd.DataFrame(0,index=range(k),columns=range(k))
S = pd.DataFrame(0,index=range(k),columns=range(1))

## Constructing $C$ and $S$
We first obtain the total of games played, wins and loss of each team to construct the diagonal of C and the vector S. 

Then we go through the results dataFrame to find the total of number of games that 2 teams play each other to construct S.

In [198]:
# calculate the number of wins and losses for each team
count_win = results["Team_win"].value_counts()
count_lose = results["Team_lose"].value_counts()

# ensure all teams are included
# need to do this because necessary because some teams might not have any wins or losses
for i in range(k):
    if i not in count_win.index:
        count_win[i] = 0
    if i not in count_lose.index:
        count_lose[i] = 0
        
# calculate the total number of matches played
total_count = count_lose + count_win

In [199]:
# iterate through all teams
for i in range(k):
    # total number of games played by the team
    total_i = total_count[i]
    # total wins
    win_i = count_win[i]
    # assign value to the S matrix, derived from the Colley's Method
    S.iloc[i] = win_i + 1 - total_i/2
    # set the diagonal element of the C matrix for the team
    C.iloc[i,i] = total_i + 2

for i in results.index:  
    team_win = results.iloc[i]["Team_win"]
    team_lose = results.iloc[i]["Team_lose"]
    C.iloc[team_win, team_lose] -= 1
    C.iloc[team_lose, team_win] -= 1

The Colley Matrix $C$ and column vector $S$ are shown as below.

In [200]:
C

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,360
0,29,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-2,0,0,0
2,0,0,30,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
3,0,0,0,32,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,-1,31,-2,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,33,0,0,0,-2
357,0,-2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,32,0,0,0
358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,32,0,0
359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,29,0


In [201]:
S

,0
0,-0.5
1,-2.5
2,2.0
3,6.0
4,0.5
...,...
356,-7.5
357,6.0
358,-7.0
359,0.5


## Results
After populating the $C$ and $S$ matrices, Colley's Method calculates the rating vector r by solving the linear system $C \widehat{x} = S$. We can achieve this through NumPy's linalg library.

In [202]:
rating_score = pd.DataFrame(np.linalg.solve(C,S), columns=['rating_score'])
rating_score

,rating_score
0,0.371811
1,0.366855
2,0.540288
3,0.840567
4,0.322280
...,...
356,0.282900
357,0.651757
358,0.401691
359,0.525445


We have obtained the ratings of every team. Since the index is ordered, we can combines the teams and rating_score DataFrames.

In [203]:
combined_rating_team = pd.concat([teams, rating_score], axis=1)
combined_rating_team

,Team_name,rating_score
0,Abilene_Chr,0.371811
1,Air_Force,0.366855
2,Akron,0.540288
3,Alabama,0.840567
4,Alabama_A&M,0.322280
...,...,...
356,Wright_St,0.282900
357,Wyoming,0.651757
358,Xavier,0.401691
359,Yale,0.525445


Then we can sort the ratings to obtain the ranks.

In [204]:
combined_rating_team.sort_values(by=["rating_score"], inplace=True, ascending=False)
combined_rating_team

,Team_name,rating_score
268,South_Carolina,1.194645
56,Connecticut,1.118797
122,Indiana,1.098393
284,Stanford,1.080892
338,Virginia_Tech,1.042407
...,...,...
202,Nicholls_St,0.012827
182,MS_Valley_St,-0.020377
190,Navy,-0.063676
282,St_Peter's,-0.072994


In [205]:
combined_rating_team.head()

,Team_name,rating_score
268,South_Carolina,1.194645
56,Connecticut,1.118797
122,Indiana,1.098393
284,Stanford,1.080892
338,Virginia_Tech,1.042407


We can see South Carolina, Connecticut, Indiana, Stanford, Virginia Tech are ranked among the top teams in the nation through Colley's Method, which definitely make sense!

# Massey's Method

Massey's Method is based on the least squares solution to a system of equations. 

Thus, if a total of $m$ games are played between $n$ teams and we construct a similar equation for each game, the result is a system of equations with $m$ equations and $n$ unknowns. 

From this, we can construct an $m \times n$ matrix, $m$ rows because there are $m$ equations, one for each game, and $n$ columns, one for each team). We call this matrix $X$. In each row, for teams $i$ and $j$ that played each other, there is a 1 in the $i$ space and a -1 in the $j$ space, indicating that these teams played each other, with team $i$ defeating team $j$, and zeros everywhere else.

To find the rating of each team, then, we might consider the linear equation $X r=y$, where $X$ is our matrix, $r$ is an $n \times 1$ vector of the ratings we are trying to find, and $y$ is an $m \times 1$ vector of the point differentials of every game. 

## Constructing $X$ and $Y$
For $Y$ we simply obtain the score difference in every match.

In [206]:
# create a new vector Y containing the point differentials for each match.
Y = results.apply(lambda row: int(row["Score_win"]) - int(row["Score_lose"]), axis=1)
Y

0        8
1       37
2       30
3        2
4       40
        ..
5369    18
5370     5
5371    10
5372    25
5373    12
Length: 5374, dtype: int64

For $X$ we then want to iterate through all rows in the results DataFrame, and modifies $X$ based on the winning and losing team IDs for each match. Specifically, we want to set the column corresponding to the winning team ID to 1 and the column corresponding to the losing team ID to -1.

In [207]:
# initialize DataFrame X with 0s in all entries
X = pd.DataFrame(0,index=results.index,columns=range(k))

In [208]:
for i in results.index:
    match = results.iloc[i]
    team_win = int(match["Team_win"])
    team_lose = int(match["Team_lose"])
    X.iloc[i, team_win] = 1
    X.iloc[i, team_lose] = -1

In [209]:
print("X has the dimension of", X.shape)
print("y has the dimentsion of", Y.shape)

X has the dimension of (5374, 361)
y has the dimentsion of (5374,)


## Computing estimated matrices

We notice that there are more equations than there are unknowns in the system, and it almost certainly does not have a solution.

Therefore we multiply by $X^{T}$ on the left on both sides and instead we consider the normal equations $X^{T} X r=X^{T} y$. We can solve for the vector $r$ in this equation as a good estimate of the ratings.

In [210]:
# calculate the matrix M by multiplying the transpose of X 
M= X.T@X
M

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,360
0,27,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-2,0,0,0
2,0,0,28,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
3,0,0,0,30,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,-1,29,-2,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,31,0,0,0,-2
357,0,-2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,30,0,0,0
358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,30,0,0
359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,27,0


In [211]:
# calculate the vector D by multiplying the transpose of the X matrix with vector Y
P = X.T@Y
P

0       63
1      -65
2      -22
3      303
4     -131
      ... 
356   -429
357    142
358   -321
359   -138
360    131
Length: 361, dtype: int64

After a few games have been played, this matrix has a rank $n-1$, which means that there are infinite solutions to this system $M r=p$. We are looking for a single solution so we can obtain the rating of each team. To fix this problem, Massey changes the last row of the matrix to a row of all ones and the corresponding entry in the point differential vector to a zero, so that the rank is no longer less than n. This system can now be solved and we can find the ratings of each team.

In [212]:
# replace the last row of the M with a row of ones, described by Massey
M.iloc[k-1] = np.ones((1,k))
M

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,360
0,27,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-2,0,0,0
2,0,0,28,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
3,0,0,0,30,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,-1,29,-2,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,31,0,0,0,-2
357,0,-2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,30,0,0,0
358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,30,0,0
359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,27,0


In [213]:
# set the last element of the D vector to 0
P.iloc[k-1] = 0
P

0       63
1      -65
2      -22
3      303
4     -131
      ... 
356   -429
357    142
358   -321
359   -138
360      0
Length: 361, dtype: int64

## Results
With the matrix $M$ and vector $D$, we can set up the linear system of equations $M r = D$ and solve for the rating vector $r$, which contains the rating scores for each team.

In [214]:
rating_score_massey = pd.DataFrame(np.linalg.solve(M,D), columns=['rating_score'])
rating_score_massey

,rating_score
0,-0.570957
1,-0.063618
2,-2.242846
3,22.027619
4,-15.232712
...,...
356,-13.942359
357,6.956286
358,-1.505157
359,-3.647176


We have obtained the ratings of every team. Since the index is ordered, we can combines the teams and rating_score_massey DataFrames.

In [215]:
combined_rating_team_massey = pd.concat([teams, rating_score_massey], axis=1)
combined_rating_team_massey

,Team_name,rating_score
0,Abilene_Chr,-0.570957
1,Air_Force,-0.063618
2,Akron,-2.242846
3,Alabama,22.027619
4,Alabama_A&M,-15.232712
...,...,...
356,Wright_St,-13.942359
357,Wyoming,6.956286
358,Xavier,-1.505157
359,Yale,-3.647176


We can then sort the ratings to obtain the ranks.

In [216]:
combined_rating_team_massey.sort_values(by=["rating_score"], inplace=True, ascending=False)
combined_rating_team_massey

,Team_name,rating_score
268,South_Carolina,45.711150
284,Stanford,35.357488
153,LSU,35.297140
56,Connecticut,35.113187
122,Indiana,35.000176
...,...,...
43,Charleston_So,-27.821825
182,MS_Valley_St,-30.403775
247,S_Carolina_St,-30.775545
46,Chicago_St,-31.013787


In [217]:
combined_rating_team_massey.head()

,Team_name,rating_score
268,South_Carolina,45.711150
284,Stanford,35.357488
153,LSU,35.297140
56,Connecticut,35.113187
122,Indiana,35.000176


We can see South Carolina, Stanford, LSU, Connecticut, and Indiana are ranked among the top teams. Four of the teams here (except LSU) are same as the predictions we have using the Colley Method, which proves accuracy.